In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import requests

In [3]:
class get_traffic_stats():
    def __init__(self, site_url):
        self.site_url = site_url
        
    def extract_stats(self):
        driver = webdriver.Firefox()
        driver.get(self.site_url)
        page = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(page, 'html5lib')
        
        ths = soup.find_all('th')
        ths = [th.text for th in ths] # Table headers
        
        tds = []
        for tr in soup.find_all('tr'):
            td = tr.find_all('td')
            if len(td) > 0:
                tds.append(td)
            
        cities, ranks = [], []
        
        for td in tds:
            cities.append(td[0].text)
            ranks.append(td[1].text)
            
        cities = cities[:10]
        global_impact_ranks = ranks[:10]
        hours_lost = ranks[10:]
        columns = ths[:2] + ths[2+1:]
        
        df = pd.DataFrame([cities, global_impact_ranks, hours_lost]).T
        df.columns = columns
        
        return df

In [4]:
get_data = get_traffic_stats('https://topauto.co.za/features/32090/10-most-traffic-congested-cities-in-south-africa/')

In [5]:
df = get_data.extract_stats()

In [6]:
df

,City,Global Impact Rank (vs 2020),Hours Lost (vs pre-Covid)
0,Cape Town,43 (95),59 hours (-53%)
1,Johannesburg,71 (139),48 hours (-36%)
2,Port Elizabeth (Gqeberha),155 (525),41 hours (+62%)
3,Pretoria,197 (154),38 hours (-42%)
4,Pietermaritzburg,315 (586),33 hours (+16%)
5,East London,353 (610),30 hours (+42%)
6,Bloemfontein,420 (898),24 hours (+51%)
7,Durban,524 (592),19 hours (-21%)
8,Vanderbijlpark,776 (972),11 hours (+84%)
9,Welkom,"1,010 (1,013)",0 hours (NA)
